# Recommendation & Accuracy 

GOALS:::

Recommendation Strategies
1. k constant: recommend with k many papers
2. k parameter: threshold on the papers

Evaluation Strategies
1. Accuracy metrics
2. Discovery-oriented metrics
"""


# Example

In [3]:
import preprocess
import random
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline
from matplotlib import rc
rc('figure', figsize=(16, 8), max_open_warning=False)
from surprise import SVD, SVDpp, NMF
from surprise import Dataset, Reader
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split, KFold


###all rights to below for these functions
###http://surprise.readthedocs.io/en/stable/FAQ.html#how-to-get-the-top-n-recommendations-for-each-user
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls




196 ['318', '313', '511', '136', '474', '114', '427', '513', '603', '479']
186 ['22', '169', '318', '195', '488', '275', '513', '64', '57', '328']
22 ['408', '169', '100', '98', '183', '64', '12', '483', '191', '182']
244 ['127', '483', '137', '134', '269', '124', '187', '408', '285', '427']
166 ['169', '174', '50', '195', '173', '79', '172', '96', '483', '484']
298 ['313', '64', '480', '515', '114', '169', '408', '487', '251', '488']
115 ['408', '169', '285', '134', '276', '168', '474', '135', '488', '923']
253 ['191', '480', '174', '169', '178', '651', '194', '320', '211', '199']
305 ['114', '513', '185', '136', '603', '205', '320', '1019', '498', '1194']
6 ['661', '657', '603', '114', '443', '1021', '1142', '251', '705', '647']
62 ['408', '169', '137', '632', '654', '187', '192', '124', '434', '430']
286 ['496', '136', '64', '134', '318', '98', '515', '589', '435', '479']
200 ['64', '12', '408', '181', '194', '427', '735', '83', '251', '178']
210 ['318', '408', '64', '285', '12', '6

461 ['408', '483', '114', '474', '493', '187', '134', '223', '127', '513']
467 ['169', '408', '318', '173', '488', '285', '483', '12', '474', '657']
468 ['408', '474', '169', '136', '83', '657', '272', '114', '187', '228']
466 ['48', '474', '657', '408', '483', '169', '615', '603', '528', '520']
472 ['242', '302', '474', '144', '512', '165', '480', '514', '479', '48']
465 ['483', '173', '515', '178', '83', '641', '661', '89', '189', '494']
463 ['64', '318', '408', '187', '114', '659', '606', '483', '178', '9']
471 ['515', '408', '318', '316', '313', '923', '513', '641', '165', '1007']
474 ['408', '272', '169', '223', '114', '251', '165', '269', '246', '512']
469 ['408', '318', '169', '114', '515', '50', '357', '190', '285', '657']
464 ['98', '408', '64', '169', '483', '272', '190', '478', '180', '516']
476 ['318', '12', '64', '408', '50', '190', '357', '603', '272', '661']
478 ['136', '603', '525', '516', '14', '528', '272', '59', '519', '709']
473 ['408', '603', '12', '318', '169', '5

In [ ]:
# First train an SVD algorithm on the movielens dataset.
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Then predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [2]:
data = Dataset.load_builtin('ml-100k')
kf = KFold(n_splits=5)
algo = SVD()

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.883757511488
0.261668474387
0.876899964652
0.243473417324
0.872788393489
0.256231358057
0.88711960368
0.259019404321
0.875114881584
0.254555161869


# Recommendation System


In [36]:
mydict = preprocess.create_random_subset_paper_paper_data(500000)
data = preprocess.create_surprise_paper_paper_data(mydict)
# trainset = data.build_full_trainset()
# testset = trainset.build_anti_testset()
#### OR
trainset, testset = train_test_split(data, test_size=.25)
####
algo = SVDpp()
algo.fit(trainset)
predictions = algo.test(testset)

k = 10
top_n = get_top_n(predictions, n=k)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings], '\n')


80d2f759-aff1-4b24-92b0-177a062e737a ['07715c33-025b-437b-85df-b094e8be59b8'] 

69b23911-83ab-4ee1-82ce-9dd0e31ffadb ['1637d86e-de13-4328-88f7-1654a2623542', 'c131bbef-7391-4b51-bb99-4a28493e886a'] 

f84dc71c-1400-45c7-bde8-556490c0f104 ['9d2f01af-98af-4e6a-966c-45378fdae334'] 

963a1cd2-126b-4774-a4f9-799e4b4e9031 ['a3506971-807c-4e2d-a1da-3fc2822de34f'] 

6c38b742-160c-463d-b6ea-c977cb7d7016 ['7625fd9b-7352-4ec4-bca5-2213ee6cb74a'] 

adcf9197-a05b-4669-90e1-d0ec298b9935 ['ae538c6d-1821-4fd2-936c-c589ae168cc3', '06e4032f-18dd-423e-b984-8b55d97862fa'] 

55e8601f-07f7-484c-aaa6-df19943794b7 ['ba0f30d2-e71c-40ae-bf9c-3c0b94170204'] 

0179dcdb-73e9-43a5-9c07-62491d502cd2 ['126c9239-0834-4453-9107-ed0a6e822f62'] 

14f7ed37-809a-4d07-bb68-ae23946d04b4 ['56d59f22-450e-423a-8813-9f001f98e7af'] 

b58cc2b6-5496-45e7-9f11-e951029e86e3 ['ce99e74e-4fc9-4ae0-8979-34b45453cb91'] 

9a9c5ed7-d319-4606-a458-c683dafc5378 ['ac752603-763a-4d13-ad60-822698754a82'] 

b133cb7d-bfc3-4e06-ad70-43ffa7242026 ['7


663b8706-7d82-4421-8ba6-660eb526e68e ['c0654e9b-2d39-40e2-bdd0-ad43b17e20bc'] 

638de886-b8ef-483f-94d8-a7d727b53650 ['ac697edc-adb5-4066-8668-28a1a8455b91'] 

d773d66c-b471-46c4-b061-82b793dee3a8 ['08e7caf0-ef92-4275-a036-17d8e20c3712'] 

612be946-fcc4-45d5-bf7a-6f1e0a38e884 ['2671b9ed-421b-4438-89d1-e20586c1345e'] 

ca3e6366-8d3c-4846-9e30-1936f6020f39 ['eefa74aa-9dbd-4201-a5c5-e1c778e0179f'] 

b2746d21-2f5e-4950-bc98-4092aea59ac4 ['a5179c18-24d1-4ad3-b8d4-5ef568f1f721', 'e00617a3-d9f3-490c-b6cc-68152c16356b'] 

a03d9ecf-1fad-4726-8d3f-57e13a75f663 ['f53d5851-471f-4864-8885-c37d51b84168'] 

ccdff6b8-0ce1-4210-acac-886712dbe543 ['edd4d15c-dc3b-4a9a-b627-6a5a124b2727'] 

ea9862bc-c5ff-4f45-aad3-a20d62482062 ['936eac79-77d0-4c64-9fb6-e55a98f0687c'] 

eaa5b6e1-916f-43d2-b7a3-ab8e809af5fb ['1d792a94-ed92-4156-a415-2047d547a5ec'] 

4b13bf3c-1339-463e-bc9c-4b130b2026fd ['fb35ccff-d0b2-470d-b6ee-3cd4cd64a0b3'] 

ed7d6406-09c9-460f-b4f4-1fec163e9456 ['62eb3e12-91a1-467b-bd50-2f07f6230402'] 



5a79528b-5386-4d8e-a410-460a0c0ed5c9 ['5657d4f2-2b91-4ebe-b308-35f834e89b12'] 

5035de48-8f51-48e5-a6af-58af01a7dd29 ['70f4dcf0-2784-4627-899c-1463988a3f52'] 

fa0e3b3b-d347-4f0e-a3ec-0c3de289b6fa ['981aeb45-56d0-4e29-9e21-a6897e12986a'] 

0fff332b-c0ea-4f4f-84b0-049fcf6077e5 ['45d86653-e0b3-4678-849f-b98137a0e2a5'] 

f584eedd-908e-46b2-93ac-68fb364fc8eb ['cddc6c44-de3c-4aad-bf26-b864f967b2ba'] 

b888484e-1ade-40a9-99f1-363cb756a390 ['7e9e6442-68d4-4080-8e35-318e80661b4d'] 

2f72d5a5-9f29-44bb-bd0a-b3cf6015762e ['a5d31c22-bdb4-4f0d-ab7b-4af2a8e37b9c'] 

39f71452-c4a8-48e6-9789-32b82853aa8d ['1723a2cd-ad59-4669-a3f2-d98a56451bb6'] 

126b8cb2-0545-4f6e-a4a1-bcdf22f3b285 ['5d970968-130c-47ed-a0e7-68405f842afb'] 

4f140e82-1bd5-4e25-82d9-b7f77f8ede7a ['905a3cdc-8ea9-47cf-8be0-ed231873ddf7'] 

457412b5-2929-4ce8-bbef-22ad107eb549 ['d2a997d8-baa7-42ac-912c-c72d34283ec3'] 

5067d190-dc33-44c7-9fd1-936c9ac34e88 ['0f70d7e0-639c-4410-a51a-7b0bac427a64'] 

6723683a-878a-48aa-994c-76edbb515b02 ['

In [47]:
precisions, recalls = precision_recall_at_k(predictions, k=1, threshold=1/2)

# Precision and recall can then be averaged over all users
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

1.0
0.97049382716
